# Roleplay actions
The purpose of this notebook is to seed the model with actions to further stimulate it to react to roleplay actions such as "meet Lorem". The actions are mixed into the model and aren't part of some kind of post processing so more is left to chance. Cause if it were guaranteed, where's the fun in that!

In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import re
from collections import Counter
import json
import sys
from textwrap import dedent
sys.path.append('..')
from config import Config
from regexes import *
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [2]:
output_path = os.path.join("..", "rp_data.txt")

# Get global sentiment!
This is the sentiment for all dragons, i.e can be used everywhere.

In [15]:
def get_global_emotions(path):
    data = []
    all_dragons = []
    with open(path, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if len(line) > 0:
                msg_match = re_msg.search(line)
                if msg_match is not None:
                    msg_from = msg_match.group(1)
                    if msg_from in Config.interactable_characters:
                        sentiment = msg_match.group(2)
                        data.append({
                            'dragon': msg_from,
                            'sentiment': sentiment
                        })
                        if not msg_from in all_dragons:
                            all_dragons.append(msg_from)
    sentiment_counter = Counter()
    for d in data:
        sentiment_counter[d['sentiment']] += 1
    result = []
    print(all_dragons)
    for sentiment in sentiment_counter:
        found_dragon_emotion = False
        for dragon in all_dragons:
            found_dragon_emotion = False
            for d in data:
                if d['dragon'] == dragon and d['sentiment'] == sentiment:
                    found_dragon_emotion = True
                    break
            if not found_dragon_emotion:
                break
        if not found_dragon_emotion:
            continue
        result.append(sentiment)
    return result
get_global_emotions("../awsw_story_input.txt")

['Nm', 'Ry', 'Lo', 'Ip', 'Br', 'An', 'Mv', 'Ad', 'Em', 'Sb', 'Dm', 'Ka', 'Rz', 'Iz', 'Zh', 'Kv']


['normal']

In [14]:
interactable_characters = Config.interactable_characters
interactable_characters_inv = { v: k for k, v in interactable_characters.items() }

class ActionReaction():
    def __init__(self, category, actions, reactions, characters, generate_lower_case = True, about_character = None, compatible_emotions = None):
        self.category = category
        self.actions = actions
        self.reactions = reactions
        self.characters = characters
        self.generate_lower_case = generate_lower_case
        self.about_character = about_character
        self.compatible_emotions = compatible_emotions
        
    def to_commands(self):
        command_lines = []
        for action in self.actions:
            for reaction_idx, reaction in enumerate(self.reactions):
                for character_code, character_data in self.characters.items():
                    scenes = ['%lastscene']
                    if 'scenes' in character_data:
                        scenes = character_data['scenes']
                        if type(scenes) is not list:
                            scenes = [scenes]
                            
                    if not character_code in ['m', 'c']:
                        # Needs to have emotion(s)
                        if 'emotions' in character_data:
                            emotions_list = character_data['emotions']
                            emotions = []
                            for el in emotions_list:
                                if reaction_idx in el['for_message']:
                                    emotions += el['emotions']
                        else:
                            # The only emotion that appears in all dragons
                            emotions = ['normal']
                    else:
                        emotions = [None] # To trigger 1 loop
                            
                    for emotion in emotions:
                        for scene in scenes:      
                            cmd_json = {
                                'category': self.category
                            }
                            if self.about_character is not None:
                                cmd_json['about_character'] = self.about_character.replace("%character_code", character_code)
                            if self.compatible_emotions is not None:
                                cmd_json['compatible_emotions'] = self.compatible_emotions
                            character_name = character_code
                            if character_code in interactable_characters:
                                character_name = interactable_characters[character_code]
                            possible_emotion_str = ""
                            if emotion is not None:
                                possible_emotion_str = f"{emotion} "
                            cmd_json['cmd'] = f'<p><msg>c "{action.replace("%character", character_name)}"<d><scn>{scene}<msg>{character_code} {possible_emotion_str}"{reaction}"'
                            command_lines.append(json.dumps(cmd_json))
                            if self.generate_lower_case:
                                lower1 = f'<p><msg>c "{action.replace("%character", character_name).lower()}"<d><scn>{scene}<msg>{character_code} {possible_emotion_str}"{reaction}"'
                                cmd_json['cmd'] = lower1
                                lower2 = f'<p><msg>c "{action.lower().replace("%character", character_name).lower()}"<d><scn>{scene}<msg>{character_code} {possible_emotion_str}"{reaction}"'
                                command_lines.append(json.dumps(cmd_json))
                                if lower1 != lower2:
                                    cmd_json['cmd'] = lower2
                                    command_lines.append(json.dumps(cmd_json))
        return command_lines

# Load FLAN-T5XXL! ⚙️
We are going to use this free alternative to ChatGPT to enhance the dragons fighting and talking skills!

In [3]:
class ActualOpenAIChatGPT:
    def __init__(self):
        model_id = "lmsys/vicuna-7b-v1.5-16k"
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            load_in_8bit=True, 
            device_map='auto'
        )

    def execute(self, instructions: str, **kwargs) -> str:
        input_ids = self.tokenizer(instructions, return_tensors="pt").input_ids.to("cuda")
        outputs = self.model.generate(input_ids, do_sample=True, **kwargs)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
chat_gpt = ActualOpenAIChatGPT()

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.37s/it]
/Projects/Personal/Endless-AWSW/Research/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/Projects/Personal/Endless-AWSW/Research/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initiali

In [105]:
character_details = {
    "Ry": """
    Name: Remy
    Description: Remy is the first dragon you meet upon your arrival in the dragons' world. He is one of the five main cast with their own routes and shows up throughout the main story. He works as the librarian and scholar at the Ministry of Culture and Arts in the town.
    Remy is a friendly and polite dragon, eager to do his job well and follow his duties to the letter. However he has endured terrible hardships which have left him empty, depressed and anxious, also feeling bullied by his superior Emera, thinking she has hired him just for her own entertainment due to his clumsiness and poor suitability for the job.
    """,
    "Lo": """
    Name: Lorem
    Description: Lorem is a delivery flyer for the town's post office. When he delivers a letter to you on your first day he requests an interview with you to clear up some media misconceptions about humans. Lorem is adventurous, determined, cheerful, and upbeat.

He also shows a shy side when asking out the protagonist in order to get to know more about humans so he can get valuable data for his videogame, but quickly switchs to his normal self after a while. Lorem loves adventure and shows no fear when doing things on his own, such as treasure hunting.

However, like everyone else, Lorem too can get stressed and feel other emotions, such as sadness, anger or anxiety, drawing and arts are some of his favorite hobbies to calm down and pass time.
"""
}

def improvised_rp_action(character_code: str, category):
    char_info = dedent(character_details[character_code]).strip()
    prompt = f"USER: Create a new action rp (like \"Drink with <name>\", \"Lick <name>\", \"Grab <name>'s hand and kiss it\") between 2 to 10 words for this character:\n{char_info}\nOnly reply with the new action between double quotes (\"). ASSISTANT: \""
    result = chat_gpt.execute(prompt, temperature=1.0, top_k=50)
    action = result[len(prompt):-1]
    print(f"action: {action}")
    prompt = f"USER: Reply (max 3 sentences or 100 words) as the following dragon:\n{char_info}\nto the following action: \"{action}\". Only write the reply from the dragons perspective, and use only the character details nessecary.\nASSISTANT: "
    result = chat_gpt.execute(prompt, temperature=1.0, top_k=50)
    print(result)
    return result
improvised_rp_action('Lo', 's')

action: Meet with Lorem
USER: Reply (max 3 sentences or 100 words) as the following dragon:
Name: Lorem
    Description: Lorem is a delivery flyer for the town's post office. When he delivers a letter to you on your first day he requests an interview with you to clear up some media misconceptions about humans. Lorem is adventurous, determined, cheerful, and upbeat.

He also shows a shy side when asking out the protagonist in order to get to know more about humans so he can get valuable data for his videogame, but quickly switchs to his normal self after a while. Lorem loves adventure and shows no fear when doing things on his own, such as treasure hunting.

However, like everyone else, Lorem too can get stressed and feel other emotions, such as sadness, anger or anxiety, drawing and arts are some of his favorite hobbies to calm down and pass time.
to the following action: "Meet with Lorem". Only write the reply from the dragons perspective, and use only the character details nessecary.

'USER: Reply (max 3 sentences or 100 words) as the following dragon:\nName: Lorem\n    Description: Lorem is a delivery flyer for the town\'s post office. When he delivers a letter to you on your first day he requests an interview with you to clear up some media misconceptions about humans. Lorem is adventurous, determined, cheerful, and upbeat.\n\nHe also shows a shy side when asking out the protagonist in order to get to know more about humans so he can get valuable data for his videogame, but quickly switchs to his normal self after a while. Lorem loves adventure and shows no fear when doing things on his own, such as treasure hunting.\n\nHowever, like everyone else, Lorem too can get stressed and feel other emotions, such as sadness, anger or anxiety, drawing and arts are some of his favorite hobbies to calm down and pass time.\nto the following action: "Meet with Lorem". Only write the reply from the dragons perspective, and use only the character details nessecary. \nASSISTANT: 🌟

# Character actions
These actions relate to characters, each action will be paired to a character response.

In [ ]:
actions = [
    ActionReaction('meeting', [
        "Meet with %character",
        "Visit %character",
        "Go to %character",
    ], [
        "Hey [player_name]!",
        "Hey [player_name]! How are you?",
        "Oh, [player_name], I didn't expect you to arrive so early.",
        "Oh, [player_name], I wasn't expecting visitors.",
    ], 
    {
        'Ry': {
            'scenes': 'remyapt', 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['smile', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['shy', 'normal'] },
            ]
        },
        'Lo': {
            'scenes': 'loremapt', 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['happy', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['shy', 'normal'] },
            ]
        },
        'Ip': {
            'scenes': 'loremapt', 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['happy', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['think', 'normal'] },
            ]
        },
        'Br': {
            'scenes': 'bare', 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['happy', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['think', 'normal'] },
            ]
        },
        'Em': {
            'scenes': 'emeraroom', 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['laugh', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['ques', 'normal'] },
            ]
        },
        'Ka': {
            'scenes': ['park3', 'park2', 'park1'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['smile', 'excited', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['exhausted', 'normal'] },
            ]
        },
        'Rz': {
            'scenes': ['park3', 'park2', 'park1'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['amused', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['annoyed', 'normal'] },
            ]
        },
        'Rz': {
            'scenes': ['park3', 'park2', 'park1'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['amused', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['annoyed', 'normal'] },
            ]
        },
        'Kv': {
            'scenes': ['park3', 'park2', 'park1'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['ramble', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['face', 'normal'] },
            ]
        },
        'Zh': {
            'scenes': ['bare'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['smile', 'normal'] },
                { 'for_message': [2, 3], 'emotions': ['shy', 'normal'] },
            ]
        },
        'Mv': {
            'scenes': ['park3', 'park2', 'park1'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['normal', 'nice'] },
                { 'for_message': [2, 3], 'emotions': ['shy', 'think', 'normal'] },
            ]
        },
        'An': {
            'scenes': ['corridor'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['normal', 'smirk'] },
                { 'for_message': [2, 3], 'emotions': ['sad', 'think', 'face'] },
            ]
        },
        'Ad': {
            'scenes': ['adineapt'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['normal', 'giggle'] },
                { 'for_message': [2, 3], 'emotions': ['think', 'face'] },
            ]
        },
        'Sb': {
            'scenes': ['office'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['normal', 'smile'] },
                { 'for_message': [2, 3], 'emotions': ['brow', 'normal'] },
            ]
        },
        'Nm': {
            'scenes': ['fac1', 'cafe', 'ecknaomiapt03', 'ecknaomiapt01'], 
            'emotions': [
                { 'for_message': [0, 1], 'emotions': ['normal', 'smile'] },
                { 'for_message': [2, 3], 'emotions': ['shy', 'normal'] },
            ]
        },
    }, about_character="%character_code"),
    ActionReaction('navigation', [
        "Leave",
        "Go to park",
    ], [
        "I walked back to the park",
        "After a long stroll, I managed to end up at the park"
    ], 
    {
        'm': {'scenes': 'park1'},
    }),
    ActionReaction('dancing', [
        "Dance",
        "Dance to the music"
    ], [
        "%lastcharacter starts dancing with me, their body moving in perfect sync with the beat",
        "%lastcharacter joins in, their dance style complementing mine",
        "We dance together, lost in the music and the moment"
    ], 
    {
        'm': {'scenes': '%lastscene'},
    }),
    ActionReaction('navigation', [
        "Leave",
        "Go to the store",
        "Go to store",
    ], [
        "I opened the door and stepped in the store",
        "The store was rather quiet",
        "The store was rather crowded"
    ], 
    {
        'm': {'scenes': 'store'},
    }),
    ActionReaction('fighting', [
        "Fight",
        "Attack"
    ], [
        "%lastcharacter barely avoids my attack and fell, but managed to get up and quickly punch me in the face, a soaring pain quickly came over my face",
        "%lastcharacter dodges my attack and comes rushing towards me",
        "I didn't hesitate and kicked %lastcharacter right in the stomach"
    ], 
    {
        'm': {'scenes': '%lastscene'},
    }),
    ActionReaction('fighting_addressed', [
        "Kick",
        "Punch",
        "Bite",
    ], [
        "Wtf you think you doing?",
        "Think you can hurt me?",
        "Stop it now or I'll fuck you up",
        "Holy shit!"
    ], 
    {
        '%lastcharactercode': {
            'scenes': '%lastscene'
        }
    }, compatible_emotions = ['angry', 'rage', 'stern', 'hurt']),
    ActionReaction('calling_out', [
        "Hey %character! Over here!",
        "Hello %character",
        "%character? Is that you?",
    ], [
        "Hey [player_name]!",
        "Hey [player_name]! How are you?",
        "[player_name]? What are you doing here?",
    ], 
    {
        k: {'scenes': '%lastscene'} for k, v in interactable_characters.items()
    }, 
    compatible_emotions = ['smile', 'shy', 'giggle', 'nice'],
    about_character="%character_code")
]

character_metadata = {
    'Lo': {
        'gender': 'm'
    },
    'Ad': {
        'gender': 'f'
    },
    'An': {
        'gender': 'f'
    },
    'Br': {
        'gender': 'm'
    },
    'Ry': {
        'gender': 'm'
    },
    'Ka': {
        'gender': 'm'
    },
    'Sb': {
        'gender': 'm'
    },
    'Kv': {
        'gender': 'm'
    },
    'Ip': {
        'gender': 'm'
    },
    'Em': {
        'gender': 'f'
    },
    'Zh': {
        'gender': 'm'
    },
    'Nm': {
        'gender': 'f'
    }
}

pronouns_mapping = {
    'm': ['he', 'him', 'He'],
    'f': ['she', 'her', 'She']
}
    
for k in character_metadata:
    character = character_metadata[k]
    pronouns = pronouns_mapping[character['gender']]
    name = Config.interactable_characters[k]
    actions.append(ActionReaction('ask_about', [
        f"What do you think of {name}?"
    ], [
        f"I think {pronouns[0]}'s cute.",
        f"I dont like {pronouns[1]} as well as before.",
        f"I like {pronouns[1]}.",
        f"{pronouns[2]} is a bit odd.",
        f"{pronouns[2]} is a bit interesting.",
        f"I think {pronouns[0]}'s funny.",
        f"I think {pronouns[0]}'s ugly.",
        f"I think {pronouns[0]}'s good looking.",
        f"I think {pronouns[0]} is funny.",
        f"I think {pronouns[0]} is ugly.",
        f"I think {pronouns[0]} is good looking.",
    ], 
    {
        '%lastcharactercode': {
            'scenes': '%lastscene'
        }
    }, generate_lower_case = False, about_character=k))
    actions.append(ActionReaction('mention', [
        f"I was with {name} today"
    ], [
        "Very nice",
        f"Oh, what did {pronouns[0]} say?",
        f"What did you think of {pronouns[1]}?",
        "That's cool",
        "Interesting. What did you two do?",
        "What happened?"
    ], 
    {
        '%lastcharactercode': {
            'scenes': '%lastscene'
        }
    }, generate_lower_case = False, about_character=k))

with open(output_path, 'w') as f:
    command_lines = []
    for action in actions:
        for cmd in action.to_commands():
            f.write(cmd + "\n")